### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = tomorrow

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
6681,2023-11-15,Brasil Nbb,19:30,União Corinthians,São José,2.16,1.67,154.5,1.80,1.86,1.5,2.00,1.72
6682,2023-11-15,Brasil Nbb,20:00,R10 Score Vasco da Gama,Cerrado,1.25,3.84,147.5,1.86,1.88,-10.5,2.04,1.69
6683,2023-11-15,Eua Nba,00:00,Denver Nuggets,Los Angeles Clippers,1.57,2.63,223.5,1.82,2.11,-4.5,1.97,1.97
6684,2023-11-15,Eua Nba,00:00,Golden State Warriors,Minnesota Timberwolves,2.34,1.70,214.5,1.80,2.08,2.5,2.05,1.89
6685,2023-11-15,Eua Nba,00:30,Los Angeles Lakers,Memphis Grizzlies,1.42,3.23,224.5,1.82,2.11,-6.5,1.97,1.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6813,2023-11-15,Eua Ncaa,21:00,Fairleigh Dickinson,St. Peters,2.00,1.83,139.5,1.80,1.95,-1.5,2.10,1.67
6814,2023-11-15,Eua Ncaa,21:00,Loyola MD,Binghamton,1.71,2.23,140.5,1.80,1.95,-3.5,2.00,1.71
6815,2023-11-15,Eua Ncaa,21:00,Nicholls State,Denver,1.76,2.14,158.5,1.87,1.79,-3.5,2.09,1.66
6816,2023-11-15,Eua Ncaa,22:00,Md.-East. Shore,Longwood,4.44,1.23,137.5,1.91,1.91,8.5,2.00,1.71


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,00:00,Eua Nba,Denver Nuggets,Los Angeles Clippers,223.5,1.82,1.0000,Over
3,00:00,Eua Nba,Golden State Warriors,Minnesota Timberwolves,214.5,1.80,1.0000,Over
4,00:30,Eua Nba,Los Angeles Lakers,Memphis Grizzlies,224.5,1.82,1.0000,Over
5,21:00,Eua Nba,Washington Wizards,Dallas Mavericks,242.5,1.94,0.8939,Over
8,21:30,Eua Nba,Toronto Raptors,Milwaukee Bucks,225.5,1.93,0.9519,Over
10,23:00,Eua Nba,Phoenix Suns,Minnesota Timberwolves,227.5,1.97,0.9997,Over
13,16:30,Europa Euroliga,Baskonia,Barcelona,160.5,1.80,1.0000,Over


### Over v2 (NB)

In [9]:
# Duplicando o dataset
df_over_v2 = jogos.copy()

df_over_v2.reset_index(drop=True, inplace=True)
df_over_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v2.empty:
    mdl_over_v2 = load_model('ML/over/v2/v2_over', verbose=False)
    prev = predict_model(mdl_over_v2, df_over_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Last_CG_H'] >= 0) & (prev['Last_CG_H'] < 130)) | ((prev['Last_CG_H'] >= 183) & (prev['Last_CG_H'] < 1485))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V2 (NB) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V2 (NB) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
1,20:00,Brasil Nbb,R10 Score Vasco da Gama,Cerrado,147.5,1.86,1.0000,Over
5,21:00,Eua Nba,Washington Wizards,Dallas Mavericks,242.5,1.94,0.8716,Over
7,21:30,Eua Nba,Philadelphia 76ers,Boston Celtics,222.5,1.85,0.6151,Over
11,16:00,Europa Euroliga,Alba Berlin,Lyon-Villeurbanne,166.5,1.91,0.9542,Over
14,16:30,Europa Euroliga,Bayern,Partizan,162.5,1.87,0.6891,Over
17,14:30,Europa Eurocopa,Cedevita Olimpija,Hamburg,173.5,1.87,0.7113,Over
19,17:00,Europa Eurocopa,Gran Canaria,Slask Wroclaw,164.5,1.86,1.0000,Over
26,20:15,Chile Copa Do Chile,Ancud,Puerto Montt,152.5,1.83,0.9962,Over
32,08:35,China Cba,Sichuan,Jilin,199.5,1.81,0.5855,Over
33,09:00,China Cba,Xinjiang,Shenzhen,195.5,1.85,0.5313,Over


### Over v3 (NB)

In [10]:
# Duplicando o dataset
df_over_v3 = jogos.copy()

df_over_v3.reset_index(drop=True, inplace=True)
df_over_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v3.empty:
    mdl_over_v3 = load_model('ML/over/v3/v3_over', verbose=False)
    prev = predict_model(mdl_over_v3, df_over_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V3 (NB) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V3 (NB) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
5,21:00,Eua Nba,Washington Wizards,Dallas Mavericks,242.5,1.94,0.8870,Over
6,21:30,Eua Nba,Atlanta Hawks,New York Knicks,229.5,1.84,0.6470,Over
7,21:30,Eua Nba,Philadelphia 76ers,Boston Celtics,222.5,1.85,0.6466,Over
8,21:30,Eua Nba,Toronto Raptors,Milwaukee Bucks,225.5,1.93,0.8856,Over
10,23:00,Eua Nba,Phoenix Suns,Minnesota Timberwolves,227.5,1.97,0.8948,Over
11,16:00,Europa Euroliga,Alba Berlin,Lyon-Villeurbanne,166.5,1.91,0.8092,Over
12,16:15,Europa Euroliga,Panathinaikos,Zalgiris Kaunas,157.5,1.83,0.5987,Over
14,16:30,Europa Euroliga,Bayern,Partizan,162.5,1.87,0.7623,Over
19,17:00,Europa Eurocopa,Gran Canaria,Slask Wroclaw,164.5,1.86,0.5304,Over
24,19:30,Chile Copa Do Chile,U. De Concepcion,Espanol de Talca,146.5,1.83,0.8859,Over
